#### Análise exploratória inicial do dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
# Leia o arquivo csv
df_train = pd.read_csv('train.csv', index_col=[0])
df_test = pd.read_csv('test.csv', index_col=[0])
# Informações sobre o dataset
print(df_train.info())
print("\n")
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6666 entries, 0 to 6665
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  6666 non-null   object
 1   label   6666 non-null   object
dtypes: object(2)
memory usage: 156.2+ KB
None


<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 6666 to 7665
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
dtypes: object(1)
memory usage: 15.6+ KB
None


In [3]:
# Mostrar as primeiras linhas
df_train.head()

,review,label
id,,
0,<p></p>not worth the money,good
1,Love magazines full of non important informati...,good
2,I have been subscribing to this magazine for y...,good
3,I was amazed at how quickly this came in the m...,good
4,...<br>I can't get Hearst to extend my subscri...,good


In [4]:
## Pré-processamento dos dados

# Verificar a existência de dados ausentes: df.isnull().sum()
print("Treino:")
print(df_train.isnull().sum())
print("\n")
print("Teste:")
print(df_test.isnull().sum())

Treino:
review    0
label     0
dtype: int64


Teste:
review    0
dtype: int64


In [5]:
## Análise exploratória dos dados

# Verificar os tipos de classes para classificação
print("\nClasses para classificação:")
print(df_train['label'].unique())


Classes para classificação:
['good' 'bad']


#### Data Cleaning e Data Normalization

- Palavras irrelevantes (StopWords), tags HTML, caracteres especiais e pontuação serão removidos. 
    - Obs: StopWords são palavras que não trazem significado à nossa frase, como "eu, sou, todos, meus, seus, nossos, deles, era...".
- Os textos serão transformados em letras minúsculas.

In [6]:
# Verificação de duplicatas
print("\nDuplicatas:")
print(df_train.duplicated().sum())



Duplicatas:
0


In [7]:
'''
# Preprocessamento dos dados
nltk.download('stopwords')
stopwords = stopwords.words('english')

def preprocess_text(text):
    # Remoção de Tags HTML
    TAG_RE = re.compile(r'<[^>]+>')
    text = TAG_RE.sub('', text)
    
    # Remoção de caracteres especiais
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)

    # Remover todos single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remover espaços em branco no início e no final
    text = text.strip()

    # Remover espaços em branco duplicados entre palavras para espaçamento simples
    text = re.sub('\s+', ' ', text)
    
    # Colocar texto em lowercase
    text = text.lower()
    
    # Remoção de Stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords)
    
    return text

df_test['review'] = df_test['review'].apply(preprocess_text)
df_train['review'] = df_train['review'].apply(preprocess_text)
df_train.head()
'''


"\n# Preprocessamento dos dados\nnltk.download('stopwords')\nstopwords = stopwords.words('english')\n\ndef preprocess_text(text):\n    # Remoção de Tags HTML\n    TAG_RE = re.compile(r'<[^>]+>')\n    text = TAG_RE.sub('', text)\n    \n    # Remoção de caracteres especiais\n    pattern = r'[^a-zA-z0-9\\s]'\n    text = re.sub(pattern, '', text)\n\n    # Remover todos single characters\n    text = re.sub(r'\\s+[a-zA-Z]\\s+', ' ', text)\n\n    # Remover espaços em branco no início e no final\n    text = text.strip()\n\n    # Remover espaços em branco duplicados entre palavras para espaçamento simples\n    text = re.sub('\\s+', ' ', text)\n    \n    # Colocar texto em lowercase\n    text = text.lower()\n    \n    # Remoção de Stopwords\n    text = ' '.join(word for word in text.split() if word not in stopwords)\n    \n    return text\n\ndf_test['review'] = df_test['review'].apply(preprocess_text)\ndf_train['review'] = df_train['review'].apply(preprocess_text)\ndf_train.head()\n"

#### Pré-Processamento

Aplicação de `Lemmatization`, de modo a reduzir as palavras à sua forma base, levando em consideração a estrutura linguística da palavra e consulta um dicionário para encontrar a forma canônica (lema) da palavra. Isso garante que as palavras raiz resultantes sejam reais e tenham significado.
- Por exemplo: as palavras "correr", "corrida" e "correndo" serão transformandas em "correr".

A decisão de aplicar o `CountVectorizer` e o `TfidfTransformer` antes de dividir o conjunto de dados em treinamento e teste (usando train_test_split) visa:

- Garantir que o pré-processamento seja aplicado de forma consistente a todos os documentos, independentemente de serem parte do conjunto de treinamento ou de teste.

In [8]:
'''
# Lemmatization
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in text.split()])

df_test.review = df_test.review.apply(lemmatize_text)
df_train.review = df_train.review.apply(lemmatize_text)
df_train.head()
'''

'\n# Lemmatization\nlemmatizer = WordNetLemmatizer()\nnltk.download(\'wordnet\')\n\ndef lemmatize_text(text):\n    return " ".join([lemmatizer.lemmatize(w) for w in text.split()])\n\ndf_test.review = df_test.review.apply(lemmatize_text)\ndf_train.review = df_train.review.apply(lemmatize_text)\ndf_train.head()\n'

In [9]:
# CountVectorizer
cv = CountVectorizer()
X_train_cv = cv.fit_transform(df_train.review)
X_df_test_cv = cv.transform(df_test.review)

In [10]:
# TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_cv)
X_df_test_tfidf = tfidf_transformer.transform(X_df_test_cv)

#### Separação dos datasets de Treino e Teste

In [11]:
## Preparação dos dados de Treino e Teste

# Separar os dados de treino em:
# X -> Conjunto de reviews após o pré-processamento
X = X_train_cv
# y -> Conjunto de classes/rótulos/Classificação
y = df_train['label']


In [12]:
'''
    X_train:  O conjunto de reviews de treinamento.
    X_test:   O conjunto de reviews de teste. São os dados que serão usados para fazer a predição a fim de avaliar a acurácia do modelo.
    y_train:  O vetor de rótulos de treinamento.
    y_test:   O vetor de rótulos de teste. É o gabarito que será usado para avaliar a acurácia do modelo.
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

# Converter a matriz esparsa (X_train e X_test) em uma matriz densa usando toarray()
X_train = X_train.toarray()
X_test = X_test.toarray()

In [13]:
# Verificar se o dataset de treino está balanceado entre as classes disponíveis para evitar viés
print("Quantidade nominal de amostras por classe:")
print(y_train.value_counts())
print("\n")
# Proporção de amostras por classe em %
print("Quantidade percentual de amostras por classe:")
print(y_train.value_counts(normalize=True))

Quantidade nominal de amostras por classe:
label
good    2666
bad     2666
Name: count, dtype: int64


Quantidade percentual de amostras por classe:
label
good    0.5
bad     0.5
Name: proportion, dtype: float64


#### Modelos gerados

In [14]:
# Pipeline para o classificador Naive Bayes (GaussianNB)
nb_model = GaussianNB()             # Intancia do modelo Naive Bayes
nb_model.fit(X_train, y_train)      # Treinamento do modelo com os dados de treino
nb_pred = nb_model.predict(X_test)  # Predição do modelo com os dados de teste

# Avaliação do modelo Naive Bayes (accuracy_score, f1_score, precision_score, recall_score)
nb_accuracy = accuracy_score(y_test, nb_pred)
nb_f1 = f1_score(y_test, nb_pred, average='macro')
nb_precision = precision_score(y_test, nb_pred, average='macro')
nb_confusion_matrix = confusion_matrix(y_test, nb_pred)
nb_recall = recall_score(y_test, nb_pred, average='macro')

print("==== Naive Bayes ====")
print("Acurácia: ", nb_accuracy)
print("F1-Score: ", nb_f1)
print("Precisão: ", nb_precision)
print("Confusion Matrix:")
print(nb_confusion_matrix)
print("Recall: ", nb_recall)

==== Naive Bayes ====
Acurácia:  0.5322338830584707
F1-Score:  0.48997242816003916
Precisão:  0.548214219673002
Confusion Matrix:
[[163 504]
 [120 547]]
Recall:  0.5322338830584707


In [15]:
# Pipeline para o classificador KNN
knn_model = KNeighborsClassifier()      # Intancia do modelo KNN
knn_model.fit(X_train, y_train)         # Treinamento do modelo com os dados de treino
knn_pred = knn_model.predict(X_test)    # Predição do modelo com os dados de teste

# Avaliação do modelo KNN (accuracy_score, f1_score, precision_score, recall_score)
knn_accuracy = accuracy_score(y_test, knn_pred)
knn_f1 = f1_score(y_test, knn_pred, average='macro')
knn_precision = precision_score(y_test, knn_pred, average='macro')
knn_confusion_matrix = confusion_matrix(y_test, knn_pred)
knn_recall = recall_score(y_test, knn_pred, average='macro')

print("==== KNN ====")
print("Acurácia: ", knn_accuracy)
print("F1-Score: ", knn_f1)
print("Precisão: ", knn_precision)
print("Confusion Matrix:")
print(knn_confusion_matrix)
print("Recall: ", knn_recall)

==== KNN ====
Acurácia:  0.5787106446776612
F1-Score:  0.5766083852895998
Precisão:  0.5803056050525968
Confusion Matrix:
[[339 328]
 [234 433]]
Recall:  0.5787106446776612


In [16]:
# Pipeline para o classificador Árvore de Decisão
dt_model = DecisionTreeClassifier()     # Intancia do modelo Árvore de Decisão
dt_model.fit(X_train, y_train)          # Treinamento do modelo com os dados de treino
dt_pred = dt_model.predict(X_test)      # Predição do modelo com os dados de teste

# Avaliação do modelo Árvore de Decisão (accuracy_score, f1_score, precision_score, recall_score)
dt_accuracy = accuracy_score(y_test, dt_pred)
dt_f1 = f1_score(y_test, dt_pred, average='macro')
dt_precision = precision_score(y_test, dt_pred, average='macro')
dt_confusion_matrix = confusion_matrix(y_test, dt_pred)
dt_recall = recall_score(y_test, dt_pred, average='macro')

print("==== Árvore de Decisão ====")
print("Acurácia: ", dt_accuracy)
print("F1-Score: ", dt_f1)
print("Precisão: ", dt_precision)
print("Confusion Matrix:")
print(dt_confusion_matrix)
print("Recall: ", dt_recall)

==== Árvore de Decisão ====
Acurácia:  0.8335832083958021
F1-Score:  0.8335798417550799
Precisão:  0.8336102038201383
Confusion Matrix:
[[553 114]
 [108 559]]
Recall:  0.8335832083958021


In [17]:
# Pipeline para o classificador Random Forest
rf_model = RandomForestClassifier()     # Intancia do modelo Random Forest
rf_model.fit(X_train, y_train)          # Treinamento do modelo com os dados de treino
rf_pred = rf_model.predict(X_test)      # Predição do modelo com os dados de teste

# Avaliação do modelo Random Forest (accuracy_score, f1_score, precision_score, recall_score)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_f1 = f1_score(y_test, rf_pred, average='macro')
rf_precision = precision_score(y_test, rf_pred, average='macro')
rf_confusion_matrix = confusion_matrix(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred, average='macro')

print("==== Random Forest ====")
print("Acurácia: ", rf_accuracy)
print("F1-Score: ", rf_f1)
print("Precisão: ", rf_precision)
print("Confusion Matrix:")
print(rf_confusion_matrix)
print("Recall: ", rf_recall)

==== Random Forest ====
Acurácia:  0.7091454272863568
F1-Score:  0.7088836844106379
Precisão:  0.7099003133396047
Confusion Matrix:
[[493 174]
 [214 453]]
Recall:  0.7091454272863569


#### Predição com base nos modelos gerados

In [18]:
# Predição dos modelos undersampling com os dados de teste
#X_df_test = X_df_test_tfidf.toarray()
X_df_test = X_df_test_cv.toarray()


# Predição teste Naive Bayes
nb_pred_test = nb_model.predict(X_df_test)

# Predição teste KNN
knn_pred_test = knn_model.predict(X_df_test)

# Predição teste Árvore de Decisão
dt_pred_test = dt_model.predict(X_df_test)

# Predição teste Random Forest
rf_pred_test = rf_model.predict(X_df_test)

In [19]:
def create_submission_file(predictions, test_df, submission_file_name="submission.csv"):
    submission_df = pd.DataFrame({'id': test_df.index, 'Target': predictions})
    submission_df.to_csv(submission_file_name, index=False)
    print(f"Submission file '{submission_file_name}' created successfully.")

In [20]:
create_submission_file(dt_pred_test, df_test)

Submission file 'submission.csv' created successfully.
